In [7]:
import pandas as pd 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse 

In [8]:
df_movie=pd.read_csv('ratings.csv')

In [9]:
df_movie

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [10]:
Utility_Maxtrix=df_movie.pivot_table(index='movieId',columns='userId',values='rating')

In [11]:
Utility_Maxtrix

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,NaN,NaN,4.0,NaN,4.5,NaN,NaN,NaN,...,4.0,NaN,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,NaN,...,NaN,4.0,NaN,5.0,3.5,NaN,NaN,2.0,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193583,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
Utility_Maxtrix=np.array(Utility_Maxtrix)

In [49]:
def Normalized_Maxtrix(arr):
    arr_temp=np.column_stack(arr)
    Avg_columns = np.zeros(arr_temp.shape[0])
    for i in range(arr_temp.shape[0]):
        Avg_columns[i] = round(np.nanmean(arr_temp[i]),2)
    for i in range(len(arr)):
        for j in range(len(arr[0])):
            arr[i][j]= arr[i][j] - Avg_columns[j]
            if np.isnan(arr[i][j]):
                arr[i][j]=0
    A_sparse=sparse.csr_matrix(arr)  
    return A_sparse

In [50]:
example_rating = np.array([[2,np.nan,np.nan,5,1,np.nan]  ### tesst
                           ,[np.nan,np.nan,3,4,3,1]
                           ,[3,1,3,np.nan,5,np.nan],
                           [np.nan,4,np.nan,np.nan,5,3]
                           ,[np.nan,1,2,3,np.nan,np.nan]
                           ,[1,np.nan,2,np.nan,2,np.nan]])

In [54]:
Avg_col=Normalized_Maxtrix(Utility_Maxtrix)